In [34]:
import math
import pandas as pd
from collections import Counter

In [35]:
with open('Data\categories.txt') as f:
    content = f.readlines()
content = [x.strip() for x in content] 
content = [x.split(';') for x in content]

#Initialize minimum support count
minsup_pct=0.01
minsup = math.floor(len(content)*0.01)


In [36]:
#for every candidate k item set, loop through the transaction database, filter for frequent k item set
def get_frequent_set_count_k(candidate_set_k):
    frequent_set_count=[]
    for x in candidate_set_k:
        cnt=0
        for j in range(len(content)):
            if x <= set(content[j]):
                cnt+=1
        if cnt>=minsup:
            frequent_set_count.append((x,cnt))
    return frequent_set_count

In [37]:
def get_candidate_set_k(frequent_set_k_minus_1, k):
    #Generate candidate k item set from frequent k-1 itemset
    tmp_candidate_set=set([i.union(j) for i in frequent_set_k_minus_1 for j in frequent_set_k_minus_1 if len(i.union(j)) == k])
    #pruning for candidate k item set, that all k-1 child set is frequent k-1 item set - Apriori Rule
    c_candidate_set=Counter([y for y in tmp_candidate_set for x in frequent_set_k_minus_1 if x < y ])
    candidate_set_k = [x for x in c_candidate_set if c_candidate_set[x]==k]
    return candidate_set_k

In [38]:
candidate_set_1=set([y for x in content for y in x])
candidate_set_1=[frozenset([x]) for x in candidate_set_1]

final_frequent_set_count=[]
loop = 1
while True:
    if loop == 1:
        candidate_set = candidate_set_1
    else:
        frequent_set = [x[0] for x in frequent_set_count]
        candidate_set = get_candidate_set_k(frequent_set, loop)
    frequent_set_count = get_frequent_set_count_k(candidate_set)
    if len(frequent_set_count)==0:
        break
    else:
        final_frequent_set_count.extend(frequent_set_count)
    loop+=1

In [39]:
file  = open('Output/patterns.txt', 'w')
for x in final_frequent_set_count:
    line=str(x[1])+':'
    for i in x[0]:
        line=line+i+';'
    file.write(line[:-1]+'\n')